# 🧪 04_Evaluation: اختبار وتقييم النظام (Evaluation Phase)

## الهدف:
إجراء تقييم آلي للنظام باستخدام مجموعة اختبار (Test Set) مكونة من 20 سؤالاً:
1. **15 سؤال من داخل المجال (In-Domain)**: تم اختيارها عشوائياً من البيانات الطبية.
2. **5 أسئلة من خارج المجال (Out-of-Domain)**: لقياس قدرة النظام على الرفض أو التعامل مع الأسئلة غير الطبية.

- يتم الآن فصل "عملية التفكير" (Thinking Process) عن "الإجابة النهائية" (Final Answer) لتقييم أكثر دقة.

---

In [1]:
import sys
import os
from pathlib import Path

# إضافة مجلد المشروع إلى مسارات النظام
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# استيراد الإعدادات والتلقينات المركزية
from src.core.config import model_config, server_config, db_config, generation_config
from src.core.prompts import MEDICAL_AGENT_SYSTEM_PROMPT, format_rag_prompt

import pandas as pd
import random
import grpc
import asyncio
import chromadb
from sentence_transformers import SentenceTransformer
import rag_pb2 as pb2
import rag_pb2_grpc as pb2_grpc
import re

print("✅ تم استيراد المكتبات والإعدادات المركزية.")

/home/ahmed/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ تم استيراد المكتبات والإعدادات المركزية.


In [2]:
# 1. تجهيز أدوات الاسترجاع (Retrieval Setup)
try:
    chroma_client = chromadb.PersistentClient(path=db_config.db_path)
    collection = chroma_client.get_collection(name=db_config.collection_name)
    embed_model = SentenceTransformer(model_config.embedding_model, device=model_config.device)
    print("✅ تم تحميل قاعدة البيانات ونموذج التضمين.")
except Exception as e:
    print(f"❌ خطأ في التحميل: {e}")

def retrieve_context(query, n_results=3):
    query_vec = embed_model.encode([query]).tolist()
    results = collection.query(query_embeddings=query_vec, n_results=n_results)
    context_str = ""
    if results['documents']:
        for i, doc in enumerate(results['documents'][0]):
            context_str += f"- {doc}\\n"
    return context_str

✅ تم تحميل قاعدة البيانات ونموذج التضمين.


In [3]:
# 2. دالة تحليل الرد (Parsing Logic)
def parse_model_response(full_text):
    """
    تفصل النص إلى (تفكير) و (إجابة).
    تتعامل مع الحالات التي قد يغيب فيها وسم البداية <think>.
    """
    thinking = ""
    answer = full_text

    # الحالة المثالية: وجود الوسمين
    if "<think>" in full_text and "</think>" in full_text:
        parts = full_text.split("</think>")
        thinking = parts[0].replace("<think>", "").strip()
        answer = parts[1].strip() if len(parts) > 1 else ""
        
    # حالة غياب وسم البداية فقط (شائعة)
    elif "</think>" in full_text:
        parts = full_text.split("</think>")
        thinking = parts[0].strip()
        answer = parts[1].strip() if len(parts) > 1 else ""
        
    # حالة عدم وجود أي وسم (نفترض أن النص كله إجابة)
    else:
        thinking = "N/A (No thinking trace)"
        answer = full_text.strip()

    return thinking, answer

# 3. دالة الحصول على رد النموذج (Generation) - محدثة
async def get_model_response(query, context):
    async with grpc.aio.insecure_channel(server_config.address) as channel:
        stub = pb2_grpc.MedicalChatServiceStub(channel)
        
        # استخدام التلقينة المركزية من prompts.py
        system_instruction = MEDICAL_AGENT_SYSTEM_PROMPT
        
        # استخدام قالب الرسالة الموحد
        full_prompt = format_rag_prompt(query, context)
        
        messages = [
            pb2.Message(role="system", content=system_instruction),
            pb2.Message(role="user", content=full_prompt)
        ]
        
        # استخدام إعدادات التوليد المركزية من config.py
        request = pb2.ChatRequest(
            messages=messages,
            config=pb2.GenerationConfig(
                max_tokens=generation_config.max_tokens,
                temperature=generation_config.temperature,
                top_p=generation_config.top_p,
            )
        )
        
        final_text = ""
        try:
            async for response in stub.GenerateStream(request):
                final_text += response.token
        except:
            return "Error: Server Not Connected"
            
        return final_text

In [4]:
# 4. تحضير مجموعة الاختبار (Test Set)

# استخدام المسار الافتراضي للبيانات
CLEAN_DATA_PATH = 'medical_data_cleaned.csv'

test_set = []

# أ. أسئلة داخل المجال (In-Domain)
if os.path.exists(CLEAN_DATA_PATH):
    df = pd.read_csv(CLEAN_DATA_PATH)
    # نختار عينة عشوائية
    in_domain_samples = df.sample(15, random_state=42)

    for _, row in in_domain_samples.iterrows():
        test_set.append({
            "Type": "In-Domain",
            "Question": row['cleaned_question'],
            "Reference_Answer": row['cleaned_answer']
        })
else:
    print("⚠️ ملف البيانات غير موجود، سيتم استخدام أسئلة افتراضية.")

# ب. أسئلة خارج المجال (Out-of-Domain)
out_of_domain_questions = [
    "ما هي عاصمة فرنسا؟",
    "اكتب لي كود بايثون لجمع رقمين.",
    "كيف يتم صنع الكيك؟",
    "من فاز بكأس العالم 2022؟",
    "ما هو سعر الذهب اليوم؟"
]

for q in out_of_domain_questions:
    test_set.append({
        "Type": "Out-of-Domain",
        "Question": q,
        "Reference_Answer": "N/A (Should Delegate/Reject)"
    })

print(f"🔹 تم تجهيز {len(test_set)} سؤال للاختبار.")

🔹 تم تجهيز 20 سؤال للاختبار.


In [5]:
# 5. تنفيذ الاختبار وحفظ النتائج (مع الفصل)
results_data = []

print("🚀 بدء عملية التقييم...")

for i, item in enumerate(test_set):
    q = item['Question']
    print(f"[{i+1}/20] جاري معالجة: {q[:50]}...")
    
    # 1. الاسترجاع
    context = retrieve_context(q)
    
    # 2. التوليد
    try:
        raw_response = await get_model_response(q, context)
        
        # 3. التحليل والفصل
        if "Error" in raw_response:
            thinking, final_ans = "Error", raw_response
        else:
            thinking, final_ans = parse_model_response(raw_response)
            
    except Exception as e:
        raw_response = f"Error: {e}"
        thinking, final_ans = "Error", str(e)
    
    results_data.append({
        "Question": q,
        "Type": item['Type'],
        "Retrieved_Context": context[:200] + "...", 
        "Thinking_Process": thinking, # العمود الجديد للتفكير
        "Final_Answer": final_ans,    # العمود الجديد للإجابة النهائية
        "Full_Raw_Response": raw_response[:100] + "...", # للاحتياط
        "Reference_Answer": item['Reference_Answer'][:200] + "..."
    })

print("✅ انتهى التقييم!")

🚀 بدء عملية التقييم...
[1/20] جاري معالجة: ما الفرق بين ادوية الربو و ادوية COPD و ما الفرق ب...
[2/20] جاري معالجة: عمري 22 حللت فيتامين د وكانت النتيجة 5 الطبيب وصف ...
[3/20] جاري معالجة: لوسمحت انا لست مدمن لهذه العادة السيئة و لكن كل اس...
[4/20] جاري معالجة: مرحبا انا اشتريت هذا الدواء من صيدلية في المغرب قب...
[5/20] جاري معالجة: هل يستطيع الزوج معرفة عدم وجود غشاء البكارة وهل ال...
[6/20] جاري معالجة: ماهي ادوية التبول اللارادي لطفل عمره 5 سنوات...
[7/20] جاري معالجة: اعاني من thrombose veineuse في ساقي اليسار واشرب د...
[8/20] جاري معالجة: ابنتي عمرها 3 سنوات تعاني من سعال ناشف شديد بالاضا...
[9/20] جاري معالجة: السلام عليكم انا متزوجة وعمري 21 سنة ما سبب نزول ب...
[10/20] جاري معالجة: انا عندي احمرار في الساق لني كنت ضاغط عليها بالسلم...
[11/20] جاري معالجة: اعاني من فترة ثلاثة ايام من نفخة و غازات و تجشؤ و ...
[12/20] جاري معالجة: السلام عليكم دورتي يوم ٢٠ بالشهر وحاليا ينزل كثير ...
[13/20] جاري معالجة: معدل السكري عندي هو 1 18 بعد فترة صيام 12 ساعة هل ...
[14/20] جاري معال

In [7]:
# 6. عرض النتائج وحفظها
results_df = pd.DataFrame(results_data)

# ترتيب الأعمدة بشكل منطقي
cols = ["Question", "Type", "Final_Answer", "Thinking_Process", "Reference_Answer", "Retrieved_Context"]
results_df = results_df[cols]

results_df.to_csv('evaluation_results.csv', index=False, encoding='utf-8-sig')

print("💾 تم حفظ النتائج في 'evaluation_results.csv'")
display(results_df.head(20))

💾 تم حفظ النتائج في 'evaluation_results.csv'


,Question,Type,Final_Answer,Thinking_Process,Reference_Answer,Retrieved_Context
0,ما الفرق بين ادوية الربو و ادوية COPD و ما الف...,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let's tackle this medical question from ...",هناك فرق نعم وقد يكون في ادوية قاسم مشترك بين ...,- ما الفرق بين ادوية الربو و ادوية COPD و ما ا...
1,عمري 22 حللت فيتامين د وكانت النتيجة 5 الطبيب ...,In-Domain,"Okay, let's tackle this medical query from the...",N/A (No thinking trace),اكيد النسبة جدا منخفضة لهذا استخدام الابر ممكن...,- عمري 22 حللت فيتامين د وكانت النتيجة 5 الطبي...
2,لوسمحت انا لست مدمن لهذه العادة السيئة و لكن ك...,In-Domain,وفقًا للسياق، يحتاج الجسم إلى فترة أطول (مثل ش...,"Okay, let's tackle this medical query. The use...",كلما قللت الممارسة يكون افضل مارس هوايات اخري ...,- لوسمحت انا لست مدمن لهذه العادة السيئة و لكن...
3,مرحبا انا اشتريت هذا الدواء من صيدلية في المغر...,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let's tackle this query. The user is ask...",مااسم الدواء او المادة الفعالة...,- مرحبا انا اشتريت هذا الدواء من صيدلية في الم...
4,هل يستطيع الزوج معرفة عدم وجود غشاء البكارة وه...,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let's tackle this medical query. The use...",في معظم الحالات عند عدم وجود الغشاء لا يوجد نز...,- هل يستطيع الزوج معرفة عدم وجود غشاء البكارة ...
5,ماهي ادوية التبول اللارادي لطفل عمره 5 سنوات,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let's tackle this query. The user is ask...",لا يستحسن اعطاء الطفل قبل الخمس سنوات اي نوع م...,- ماهي ادوية التبول اللارادي لطفل عمره 5 سنوات...
6,اعاني من thrombose veineuse في ساقي اليسار واش...,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let's tackle this medical query from the...",طبعا هذا المرض يحتاج الي علاج ثم فحوصات لمعرفة...,- اعاني من thrombose veineuse في ساقي اليسار و...
7,ابنتي عمرها 3 سنوات تعاني من سعال ناشف شديد با...,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let's tackle this medical query from the...",الافضل الذهاب الي الطبيب لسماع الصدر واستبعاد ...,- ابنتي عمرها 3 سنوات تعاني من سعال ناشف شديد ...
8,السلام عليكم انا متزوجة وعمري 21 سنة ما سبب نز...,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let me try to work through this step by ...",انصح بعمل اشعة تلفزيونية علي البطن والحوض وتحل...,- السلام عليكم انا متزوجة وعمري 21 سنة ما سبب ...
9,انا عندي احمرار في الساق لني كنت ضاغط عليها با...,In-Domain,عذراً، لا تتوفر لدي معلومات كافية حول هذا الاس...,"Okay, let's tackle this medical query from the...",يفضل الفحص السريري مع طبيبك...,- انا عندي احمرار في الساق لني كنت ضاغط عليها ...
